<h3>Deploying Machine Learning Models</h3>

<h4>Installing Required Packages</h4>

In [1]:
%pip install flask 
%pip install fastapi

   ---------------------------------------- 0.0/101.7 kB ? eta -:--:--
   --------------- ----------------------- 41.0/101.7 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 101.7/101.7 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/97.9 kB ? eta -:--:--
   ---------------------------------------- 97.9/97.9 kB 5.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/133.3 kB ? eta -:--:--
   ---------------------------------------- 133.3/133.3 kB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/227.6 kB ? eta -:--:--
   ---------------------------- ----------- 163.8/227.6 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 227.6/227.6 kB 4.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/125.2 kB ? eta -:--:--
     ------------ ------------------------ 41.0/125.2 kB 960.0 kB/s eta 0:00:01
     -------------------------------------- 125.2/125.2 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/93.5 kB ? eta -:--:--
   ---------------------------------------- 93.5/93.5 kB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/423.9 kB ? eta -:--:--
   ---------------- ----------------------- 174.1/423.9 kB 3.5 MB/s eta 0:00:01
   ---------------------------------- ----- 368.6/423.9 kB 3.8 MB/s eta 0:00:01
   ---------------------------------------- 423.9/423.9 kB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---- ----------------------------------- 0.2/1.9 MB 6.9 MB/s eta 0:00:01
   --------- ------------------------------ 0.5/1.9 MB 5.9 MB/s eta 0:00:01
   -------------- ------------------------- 0.7/1.9 MB 5.6 MB/s eta 0:00:01
   --------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


<h4>Storing and Extracting Model</h4>

In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle

#Load dataset and split it
iris = load_iris()
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size= 0.2, random_state=42)

#Train a Logistics Regression model
model = LogisticRegression(max_iter= 200, random_state=42)
model.fit(x_train, y_train)

#Make Prediction with the runtime model 
predictions = model.predict(x_test)
print(f"Run Time Model Predictions : {predictions}")
print(f"Run Time Model Accuracy : {model.score(x_test, y_test):.2f}")

#Save the model to a file using pickle
with open("iris_logistics_regression.pkl", 'wb') as f:
    pickle.dump(model, f)

#Load the model from the file using pickle
with open("iris_logistics_regression.pkl", "rb") as f:
    loaded_model = pickle.load(f)

#Make prediction with the loaded model 
predictions = loaded_model.predict(x_test)
print(f"Extracted Model PRedictions : {predictions}")
print(f"Loaded Model Accuracy : {loaded_model.score(x_test, y_test):.2f}")


Run Time Model Predictions : [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Run Time Model Accuracy : 1.00
Extracted Model PRedictions : [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Loaded Model Accuracy : 1.00


<h4>Introducing Flask</h4>

In [6]:
from flask import Flask, request, jsonify
import joblib
import os

#Initialize flask app
app = Flask(__name__)

#Load the trained model 
model = joblib.load('iris_logistics_regression.pkl')

#Define a route for the prediction API
@app.route('/predict', methods= ['POST'])
def predict():
    data = request.get_json(force=True)
    prediction = model.predict([data['features']])
    return jsonify({'prediction' : int(prediction[0])})

#Run the flask app
if __name__ == '__main__':
    app.run(host=os.getenv('IP', '0.0.0.0'),
        port = int(os.getenv('PORT', 4444)))

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:4444
 * Running on http://192.168.1.124:4444
Press CTRL+C to quit
192.168.1.124 - - [28/Aug/2024 19:31:34] "GET / HTTP/1.1" 404 -
192.168.1.124 - - [28/Aug/2024 19:31:34] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/Aug/2024 19:36:50] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [28/Aug/2024 19:36:50] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/Aug/2024 19:47:12] "GET /predict HTTP/1.1" 405 -
192.168.1.124 - - [28/Aug/2024 19:53:02] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2024 19:57:19] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2024 19:57:27] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [28/Aug/2024 19:57:37] "POST /predict HTTP/1.1" 200 -


<h4>Deploying Models Using FastAPI</h4>

In [9]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, conlist
import joblib

#Inintialize FastAPI app 
app = FastAPI()

#Load the trained model 
model = joblib.load('iris_logistics_regression.pkl')

#Define request model
class Features(BaseModel):
    features : conlist(float, min_length=4, max_length=4) #Ensure the feature list has exactly 4 float values

#Define a route for the prediction API
@app.post('/predict')
def predict(data : Features):
    try :
        #Predict using the model 
        prediction = model.predict([data.features])
        return {'prediction' : int(prediction[0])}
    except Exception as e:
        #Handle eroor during prediction
        raise HTTPException(status_code=500, detail=str(e))